# Loading data

In [22]:
TUGRAFA_SPARK_HOST      = "spark://localhost:7077" 
TUGRAFA_SPARK_APP_NAME  = "Tugrafa"
TUGRAFA_DATA_FOLDER     = "notebook/data"

from pyspark.sql import SparkSession

configuration = SparkConf() \
    .setAppName(TUGRAFA_SPARK_APP_NAME) \
    .setMaster(TUGRAFA_SPARK_HOST) \
    .set("spark.cassandra.connection.host", "127.0.0.1") \
    .set("spark.cassandra.connection.port", 9042)

# Create spark context or connect to existing one
sparkContext = SparkSession.builder.config(configuration) \
    .getOrCreate()

AttributeError: 'SparkConf' object has no attribute '_get_object_id'

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(auth_provider = auth, contact_points = ["127.0.0.1"], port = 9042)

session = cluster.connect()
session

In [ ]:
cql_keyspace = """
CREATE KEYSPACE IF NOT EXISTS tugrafa
    WITH REPLICATION = {
        'class' : 'SimpleStrategy',
        'replication_factor' : 1
    }
;
"""

session.execute(cql_keyspace)

cql_table = """
CREATE TABLE IF NOT EXISTS tugrafa.data (
    id UUID PRIMARY KEY,
    card_id ASCII,
    swipe ASCII,
    poi TEXT
);
"""

session.execute(cql_table)

In [ ]:

from uuid import uuid4
import pandas as pd

df = pd.read_csv("data/dati_2014.csv")
df = df.iloc[:, [0, 1, 2, 4]]
print(df)

cql_insert_begin = "BEGIN BATCH;\n"
cql_insert_value = "INSERT INTO tugrafa.data (id, card_id, swipe, poi) VALUES ({UUID}, '{CARD_ID}', '{SWIPE}', '{POI}');\n"
cql_insert_end   = "APPLY BATCH;\n" 

BATCH_SIZE = 1000

cql_insert = ""
current_batch_size = 0
for (num, row_date, row_time, row_poi, row_card_id) in df.itertuples():
    
    # Create a new batch
    if current_batch_size == 0:
        cql_insert += cql_insert_begin
    
    # Insert value into batch
    cql_insert += cql_insert_value.format(UUID = uuid4(), CARD_ID = row_card_id, SWIPE = row_date, POI = row_poi)
    
    # Insert batch into database
    if current_batch_size == BATCH_SIZE:
        cql_insert += cql_insert_end
        print(cql_insert)
        
        session.execute(cql_insert)
        current_batch_size = 0
        cql_insert = ""
    
# If there are not inserted elements create the last batch
if current_batch_size != 0:
    cql_insert += cql_insert_end
    session.execute(cql_insert)



In [31]:
cql_select = """
SELECT * FROM tugrafa.data
LIMIT 5000;
"""

session.execute(cql_select).all()

[Row(id=UUID('f575181d-40e6-4ce0-8ea8-c3fd12106b91'), card_id='04DA80523F3880', poi='Arena', swipe='28-10-14'),
 Row(id=UUID('20cc226c-e276-41f5-8a2a-8dc7e2641c97'), card_id='0456404A3F3880', poi='Castelvecchio', swipe='01-09-14'),
 Row(id=UUID('b360c883-10bd-47bd-b77b-06709336c90b'), card_id='0431B04A3F3880', poi='Duomo', swipe='21-04-14'),
 Row(id=UUID('34feeff6-a7ad-42e6-9b42-8a40d8802011'), card_id='04E066523F3880', poi='Santa Anastasia', swipe='08-11-14'),
 Row(id=UUID('5bf5d668-1e5d-4be9-9799-ba3a84862f7a'), card_id='044E704A3F3880', poi='Duomo', swipe='17-05-14'),
 Row(id=UUID('25980bc4-9855-4b10-8111-7372813215e1'), card_id='0444BB523F3880', poi='Castelvecchio', swipe='02-10-14'),
 Row(id=UUID('8665048e-6c86-43c9-a496-06d32d9f7918'), card_id='04399B8ABF3181', poi='Torre Lamberti', swipe='27-04-14'),
 Row(id=UUID('13b49362-cb9a-4ad5-a5c9-7b528ea434f8'), card_id='043D56523F3880', poi='Giardino Giusti', swipe='05-11-14'),
 Row(id=UUID('c6c9d9cf-11f8-4f43-a632-f8dec9219ca5'), card_